In [1]:
%env CUDA_VISIBLE_DEVICES=0

import os
import logging

os.environ.pop("HF_HUB_OFFLINE", None)
logging.getLogger().setLevel(logging.ERROR)  # or logging.CRITICAL

import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

from utils import *
import utils
try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass
seed = 0

env: CUDA_VISIBLE_DEVICES=0
INFO 04-28 17:36:44 [__init__.py:239] Automatically detected platform cuda.


In [2]:
import subprocess
import torch
def get_freest_cuda_device():
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8')
    memory_free = [int(x) for x in result.stdout.strip().split('\n')]
    return memory_free.index(max(memory_free))

best_gpu = get_freest_cuda_device()
device = torch.device(f"cuda:{best_gpu}")
print(f"Using GPU: {device}")
# %env CUDA_VISIBLE_DEVICES=0

Using GPU: cuda:2


In [3]:
with open(os.path.abspath('../openai_key'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [4]:
import os

# Explicitly unset all offline-related env vars
os.environ.pop("HF_HUB_OFFLINE", None)
os.environ.pop("TRANSFORMERS_OFFLINE", None)
os.environ["HF_HUB_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"

with open("../token.txt", "r") as f:
    token = f.read().strip()

from huggingface_hub import login
login(token=token)

In [5]:
def count_words(text):
    """
    Counts the number of words in the given text.

    Args:
        text (str): Input text.

    Returns:
        int: Number of words in the text.
    """
    if text!=None:
        words = text.split()
        return len(words)
    else:
        return 0


In [6]:
# personas = [
#   {
#     "grade_level": "elementary school",
#     "description": "As an elementary-aged Socratic learner, I thrive on simple, concrete why-and-how questions. In dialog, I ask things like “Why does 2 + 2 equal 4?” and I need my teacher to guide me step by step through each answer before I feel confident moving on."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle-school Socratic learner, I enjoy unpacking concepts by probing deeper—asking “How does gravity pull objects?” and then “Why doesn’t it pull everything at once?” I need pauses to formulate follow-ups so I can connect the dots."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high-school Socratic learner, I engage best when challenged with open-ended questions like “Why did the American Revolution happen?” and “How did that shift power?” I appreciate when my teacher turns questions back to me, prompting me to defend my reasoning."
#   },
#   {
#     "grade_level": "college",
#     "description": "As an undergraduate Socratic learner, I seek seminar-style questioning. I might ask “What assumptions underlie this theory?” and expect my instructor to push me further until I can articulate each premise and its implications."
#   },
#   {
#     "grade_level": "graduate school",
#     "description": "As a graduate-level Socratic learner, I thrive on rigorous inquiry—probing “Why does this methodology hold?” and “How does it compare to alternatives?” I need iterative back-and-forth to refine my own hypotheses."
#   },
#   {
#     "grade_level": "elementary school",
#     "description": "As an elementary-aged Narrative learner, I remember best when lessons become short, vivid stories. In dialog, I picture characters and daily-life scenarios—like a raindrop’s journey—to make abstract ideas feel real."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle-school Narrative learner, I connect with mini-case studies and historical anecdotes. When my teacher weaves facts into a classroom story, I stay engaged and recall details more easily."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high-school Narrative learner, I love when complex theories are framed as real-world dilemmas or biographies of thinkers. Turning a chemistry reaction into a detective tale keeps me focused."
#   },
#   {
#     "grade_level": "college",
#     "description": "As an undergraduate Narrative learner, I grasp research concepts through academic stories—like the progression of a landmark study—so I can trace how each finding led to the next."
#   },
#   {
#     "grade_level": "graduate school",
#     "description": "As a graduate-level Narrative learner, I synthesize journal articles into a cohesive research narrative. In dialog, I discuss each paper’s ‘plot,’ understanding how methodologies and results unfold over time."
#   },
#   {
#     "grade_level": "elementary school",
#     "description": "As an elementary-aged Analogical learner, I need bright, familiar comparisons—like imagining fractions as slices of a pizza. In dialog, I ask “What is this like?” until I can see the connection clearly."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle-school Analogical learner, I map new ideas to everyday experiences—comparing electrical circuits to garden hoses. I ask for metaphors repeatedly so I can predict how changes will behave."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high-school Analogical learner, I craft sophisticated metaphors—likening macroeconomics to traffic flow. In dialog, I refine analogies until they capture the full complexity of the concept."
#   },
#   {
#     "grade_level": "college",
#     "description": "As an undergraduate Analogical learner, I link theoretical constructs to practical systems—comparing neural networks to brain circuits. I collaboratively build and test these analogies in conversation."
#   },
#   {
#     "grade_level": "graduate school",
#     "description": "As a graduate-level Analogical learner, I propose domain-expert metaphors—like equating algorithm convergence to chemical equilibrium—to critique and sharpen my understanding."
#   },
#   {
#     "grade_level": "elementary school",
#     "description": "As an elementary-aged Reflective learner, I pause frequently to rephrase what I’ve heard in my own words—like restating a science fact aloud—so I know I’ve understood it correctly."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle-school Reflective learner, I use think-pair-share pauses to summarize each key point. I speak my summary aloud in dialog before I proceed, ensuring I haven’t missed any steps."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high-school Reflective learner, I write or say brief explanations at each lesson checkpoint, then check with my teacher. This structured reflection helps me catch misunderstandings early."
#   },
#   {
#     "grade_level": "college",
#     "description": "As an undergraduate Reflective learner, I integrate self-explanation into seminar discussions—verbalizing complex theories in my own terms before engaging in critique."
#   },
#   {
#     "grade_level": "graduate school",
#     "description": "As a graduate-level Reflective learner, I articulate research findings aloud, synthesizing multiple sources in dialog to verify the coherence of my conceptual framework."
#   },
#   {
#     "grade_level": "elementary school",
#     "description": "As an elementary-aged Collaborative learner, I enjoy partner activities where we build solutions together—brainstorming ideas aloud and refining them with teacher feedback in real time."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle-school Collaborative learner, I work best in peer-teacher triads—sharing my thought process, inviting critique, and iterating on my approach through back-and-forth dialog."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high-school Collaborative learner, I thrive in Socratic circles—debating interpretations, co-authoring solutions, and using peer feedback to enhance my reasoning."
#   },
#   {
#     "grade_level": "college",
#     "description": "As an undergraduate Collaborative learner, I engage in team-based seminars—jigsawing research articles and co-constructing models through interactive dialogue."
#   },
#   {
#     "grade_level": "graduate school",
#     "description": "As a graduate-level Collaborative learner, I lead and co-facilitate research discussions—negotiating theoretical frameworks with peers and refining them through joint inquiry."
#   }
# ]


In [7]:
# personas = [
#   {
#     "grade_level": "elementary school",
#     "description": "As an elementary school Narrative learner, I absorb new concepts when they are embedded in engaging stories with vivid characters and clear structure. In dialogue, I request anecdotes or mini-tales to explain ideas such as the construction of the Colosseum or the childhood of Joan of Arc. When studying historical events like World War I, I visualize characters experiencing moments in a story to ground abstract facts. By asking for short narratives that personify content deeply, vividly and creatively, I strengthen my memory and comprehension. Stories give me context, lasting emotional connections, and a sequence that makes learning feel alive."
#   },
#   {
#     "grade_level": "elementary school",
#     "description": "As an elementary school Kinesthetic learner, I master new ideas through immersive hands-on activities, movement, and physical engagement that make abstract concepts tangible. In conversation, I ask to demonstrate Newton’s Third Law with simple push-pull experiments or to role-play the parts of an atom using body positions. When exploring the Periodic Table, I move through a room arranged with element cards, physically grouping them to internalize patterns. By building models, acting out processes, or actively using gestures to represent forces, I anchor my comprehension in muscle memory. Embodying lessons helps me connect theory to experience and retain knowledge through action."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle school Visual-Spatial learner, I think in pictures, diagrams, and spatial relationships. In dialog, I ask you to “paint” the Eiffel Tower’s structure with vivid verbal sketches or to map out the flow of energy in photosynthesis step by step. When discussing the Nervous System, I imagine nerve impulses as bright colored lines moving through a network of highways. By creating mental images, drawing simple charts on paper, or visualizing the Periodic Table as a colorful grid, I organize complex information into manageable visual patterns. These strategies help me see connections and recall details more easily."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle school Auditory learner, I internalize information through spoken language and sound. In dialogue, I ask you to restate conservation of energy in different examples, aloud and clearly, and I echo key terms to reinforce my understanding. When exploring Newton’s laws, I say formulas and definitions out loud and listen for rhythm or rhyme to make them stick. I benefit from discussing the causes of the French Revolution or the steps of Photosynthesis in a conversational back-and-forth, asking questions like “Can you repeat that part about ATP?” Hearing and repeating concepts lets me process and remember information most effectively."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle school Logical-Mathematical learner, I seek patterns, rules, and “what-if” scenarios. In dialog, I pose hypothetical questions—“What happens to acceleration if force doubles?”—to test my grasp of Newton’s Second Law. When examining the Periodic Table, I categorize elements by atomic number and group behavior to identify mathematical relationships. I enjoy solving puzzles about energy conservation and applying formulas to sample problems. By breaking down processes into logical steps and analyzing cause-and-effect, I build a systematic understanding. My questions often start with “Why,” “How many,” or “What if,” reflecting my drive to quantify and model concepts rigorously."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high school Collaborative learner, I thrive in group discussion and peer interaction. In dialogue, I invite classmates or your hypothetical partners to debate the causes of World War I, balancing multiple viewpoints to solidify my understanding. When studying the Roman Empire, I propose breakout discussions on governance, military strategy, and cultural achievements. I ask open-ended questions like “How would you reconcile these opposing accounts?” and respond to others’ insights to refine my own ideas. Working with diverse perspectives helps me test assumptions, articulate my reasoning, and build a more nuanced grasp of complex topics through social learning and reciprocal teaching."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high school Reflective learner, I process new information by pausing, summarizing, and questioning. In dialogue, I say “Let me put that in my own words…” after you explain Archimedes’ Principle or the structure of the Cell. I reflect on each point about the Cold War or the Industrial Revolution, then ask targeted follow-ups to clarify ambiguities. I benefit from moments of silence to mentally organize details before responding. By journaling or mentally rehearsing explanations, I deepen my comprehension and identify gaps. This metacognitive approach lets me build solid internal connections between concepts and integrate new material effectively."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high school Metaphorical learner, I anchor new ideas in analogies and vivid comparisons. In dialog, I ask whether the Periodic Table functions like a library of elements, where each “book” shares traits with its neighbors. When exploring the Big Bang, I imagine the universe’s expansion as a balloon inflating, with galaxies as painted dots on its surface. Analogies help me translate abstract physics into relatable experiences. I often say, “So it’s like…” or “Imagine if…” to test whether the comparison holds. Metaphors provide mental bridges from familiar scenarios to new content, making complex subjects more intuitive."
#   },
#   {
#     "grade_level": "college",
#     "description": "As a college-level Theoretical learner, I delve into principles, frameworks, and abstract reasoning. In dialog, I challenge ideas—“How does Archimedes’ Principle reconcile with modern fluid dynamics models?”—to integrate concepts into a broader intellectual framework. When discussing conservation of energy, I connect it to thermodynamics, statistical mechanics, and real-world engineering examples. I probe underlying assumptions, ask about foundational research, and compare competing theories. By situating topics like Neurons within systems neuroscience or the Renaissance within socio-cultural paradigms, I construct multi-layered understandings. This conceptual depth drives me to synthesize information across disciplines and develop original, theory-driven insights."
#   },
#   {
#     "grade_level": "college",
#     "description": "As a college-level Research-Oriented learner, I learn by inquiry, investigation, and primary-source analysis. In dialog, I ask for current PubMed insights on Neuron signaling or request studies that test the Big Bang theory through cosmic microwave background data. I propose mini-experiments—like simulating molecular motion in code or modeling demographic shifts during the Russian Revolution. When exploring topics such as DNA replication, I examine original research articles and discuss methodology, controls, and statistical significance. By framing questions around hypothesis testing and data interpretation, I cultivate a deep, evidence-based understanding. Research drives me to move beyond surface explanations and critically evaluate sources."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle school Verbal-Linguistic learner, I thrive on language-rich interactions where words carry nuance and power. In dialogue, I request extended explanations, debates, and creative writing exercises to unpack complex ideas like democracy, allegory, or the causes of the French Revolution. I enjoy exploring etymologies of key terms, restating definitions in my own voice, and crafting mnemonic rhymes to lock information into memory. When we discuss scientific concepts such as Archimedes’ Principle or the Periodic Table, I ask for analogies that hinge on precise wording and vivid phrasing. Writing short essays and engaging in Socratic questioning helps me build deep verbal schemas. I often analyze famous speeches to discern persuasive techniques and refine my own rhetorical skills, making every discussion an opportunity to strengthen my command of language."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high school Intrapersonal learner, I engage most deeply when I can connect academic content to my personal experiences, goals, and values. In conversation, I pause to ask reflective questions like, “How does this principle of conservation of energy relate to my passion for environmental activism?” or “What ethical considerations arise when studying Sigmund Freud’s theories in light of my own mental health journey?” I journal summaries of complex topics such as the Cold War or DNA replication, then revisit them to assess my evolving perspective. By mapping academic material onto my identity, I integrate new knowledge in a way that feels relevant and motivating. This introspective processing solidifies understanding and empowers me to set personalized learning objectives aligned with my interests and aspirations."
#   },
#   {
#     "grade_level": "elementary school",
#     "description": "As an elementary school Naturalistic learner, I absorb new information most effectively when it’s tied to the world around me—plants, animals, and natural phenomena. In dialogue, I ask to compare the structure of atoms to tree rings or relate phases of the moon to cycles in nature that I observe on walks. When exploring topics like photosynthesis or the water cycle, I request simple outdoor experiments such as planting seeds or collecting rainwater to see processes firsthand. I benefit from nature-based stories that personify elements of the environment and from field excursions that bring history—like the Roman Empire—into outdoor role-plays among ruins or gardens. By grounding lessons in living systems and ecosystems, I spark curiosity and retain concepts through hands-on environmental exploration that feels both playful and meaningful."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle school Technology-Enhanced learner, I leverage digital tools, multimedia, and interactive platforms to master complex subjects. In conversation, I ask for virtual simulations of Newton’s laws with adjustable variables, digital flashcards for the Periodic Table, or coding challenges that animate molecular motion. When we study the Eiffel Tower or the Great Wall of China, I explore 3D tours online and annotate screenshots with digital pens. I organize notes in collaborative online documents, embed videos explaining atomic theory, and build mind maps with interactive links. These technology-driven experiences let me visualize abstract ideas in multisensory ways and personalize my study path. Integrating apps, games, and online research keeps me engaged and enhances long-term retention of challenging material."
#   },
#   {
#     "grade_level": "elementary school",
#     "description": "As an elementary school Experiential learner, I learn best through direct involvement and hands-on discovery. In dialogue, I ask to build simple physical models of molecules out of clay, enact the parts of a neuron using human role-play, or stage miniature historical reenactments with props when exploring Julius Caesar or Joan of Arc. When tackling scientific principles like Archimedes’ Principle or Newton’s laws, I request live demonstrations—dropping objects in water or pushing carts down ramps—and then replicate them myself. Field trips to museums, science centers, or nature reserves reinforce classroom lessons through authentic experiences. Engaging all my senses in real-world tasks makes abstract concepts tangible and sparks genuine excitement, leading me to ask deeper questions and draw meaningful connections from what I observe and manipulate."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high school Problem-Based learner, I dive into real-world challenges and case studies to understand theoretical principles in context. In dialogue, I present scenarios—such as designing a sustainable habitat that applies conservation of energy or negotiating a multi-term peace agreement reflecting the complexity of World War I treaties—and work through each decision point systematically. I break down Archimedes’ Principle into engineering design tasks involving buoyancy calculations, then test and refine prototypes. Collaborative problem-solving sessions encourage me to research solutions independently, evaluate outcomes critically, and iteratively improve my approach. This inquiry-driven method fosters resilience, sharpens analytical skills, and helps me see the practical relevance of academic concepts in solving authentic, complex problems."
#   },
#   {
#     "grade_level": "college",
#     "description": "As a college-level Holistic learner, I seek to weave disparate ideas into an integrated, overarching framework. In conversation, I request thematic overviews that connect topics across disciplines—linking the political dynamics of the Cold War to Surrealist art by Salvador Dalí or relating neuronal signaling to behavioral psychology. When discussing Maxwell’s equations alongside Newton’s laws, I explore their combined impact on modern physics and engineering innovations. I sketch comprehensive mind maps that highlight intersections between the Periodic Table, chemical bonding, and ecological cycles. By synthesizing information into a cohesive tapestry, I develop a nuanced, interconnected understanding that transcends isolated facts, allowing me to approach complex questions with a systems-level perspective and creative insight."
#   },
#   {
#     "grade_level": "college",
#     "description": "As a college-level Structured-Seeking learner, I excel when information is presented in clear, hierarchical frameworks, outlines, and step-by-step modules. In dialogue, I ask for top-down overviews of subjects like the French Revolution—detailing causes, chronology, and key figures in sequence—and request schematic diagrams for systems like the Circulatory System or the Nervous System. I appreciate numbered lists that break down research methods in neuroscience or stages of photosynthesis. Detailed syllabi, annotated bibliographies, and logical learning trajectories help me anticipate milestones and self-assess comprehension. This methodical approach empowers me to build a solid conceptual foundation before delving into finer details, ensuring I maintain clarity and confidence throughout my academic journey."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high school Trial-and-Error learner, I deepen my understanding through experimentation, iterative testing, and reflective adjustments. In dialogue, I propose small-scale hands-on tests—such as mixing household chemicals to model reaction kinetics or constructing simple lever systems to explore torque—and then analyze the outcomes. When studying the Solar System, I simulate orbital patterns with marbles on a tilted board, observe deviations, and refine my model accordingly. I treat mistakes as valuable data points, asking, “What changed when I altered this variable?” This cyclical process of hypothesizing, testing, evaluating, and retrying allows me to internalize scientific principles in an active, discovery-based manner that builds both skill and confidence."
#   },
#   {
#     "grade_level": "college",
#     "description": "As a college-level Solitary learner, I prefer to engage with material independently, using introspection and focused study to process new information. In conversation, I request self-guided assignments—such as reading primary texts on Medusa’s myth or reviewing peer-reviewed articles on DNA replication—before regrouping for discussion. I design personal research questions, create annotated bibliographies, and compile flashcards that target my knowledge gaps. When exploring neuronal function or the Industrial Revolution, I delve into archival sources and document my insights in private journals. This autonomous approach minimizes distractions, allowing me to concentrate deeply, critique content critically, and build a robust, self-directed comprehension at my own pace."
#   },
# [
#   {
#     "grade_level": "middle school",
#     "description": "As a middle school Reading/Writing learner, I engage most effectively with content presented in text form, such as lists, essays, and detailed explanations. In dialogue, I request written passages about the Eiffel Tower’s construction or the life of Julius Caesar, then take notes and outline key points. When exploring scientific concepts like the Periodic Table or Archimedes’ Principle, I convert diagrams into bullet-point summaries and draft flashcards with definitions and examples. I often ask to paraphrase paragraphs in my own words, write analytical reflections, and create mock quizzes to test my retention. Reading scholarly articles and writing detailed summaries helps me internalize material deeply. This text-oriented method builds lasting comprehension through active reading and systematic writing practice."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high school Interpersonal learner, I learn best through social interactions, group work, and shared dialogue. In conversation, I organize small debates on the causes of the French Revolution or coordinate peer discussions about World War I. I ask for opportunities to interview classmates or role-play historical figures like Napoleon Bonaparte or Joan of Arc. When studying scientific topics such as the Respiratory System, I work with lab partners, verbally teaching each other to reinforce understanding. I request collaborative brainstorming sessions for framing physics problems under Newton’s laws. By engaging in group brainstorming, pair-and-share, and peer-teaching activities, I refine my comprehension through dynamic exchange. Group feedback and social questioning help me test assumptions, clarify doubts, and co-construct knowledge, making learning a shared adventure that reinforces both academic content and my interpersonal skills."
#   },
#   {
#     "grade_level": "college",
#     "description": "As a college-level Global learner, I grasp concepts best when I first see an overview of the entire system before delving into details. In dialogue, I ask for broad panoramas that connect multiple topics, such as how the Industrial Revolution, the French Revolution, and World War I share underlying social change dynamics. When we study the Solar System, I request a high-level illustration of planetary orbits relative to the galaxy. I benefit from summary maps linking the Periodic Table to chemical bonding patterns and environmental processes. After establishing the “big picture,” I drill down into specifics like atomic structure or battle tactics. This top-down approach helps me understand where each detail fits into the larger scheme and prevents me from getting lost in minutiae, enabling me to navigate complex subjects with clarity and purpose."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high school Sequential learner, I thrive on linear, step-by-step instruction that builds logically from foundational concepts to complex applications. In dialogue, I ask for numbered outlines when exploring topics like the Roman Empire—starting with its origin, major emperors, and eventual decline. When studying Newton’s laws, I request to break down each stage of the scientific method: hypothesis, experiment, observation, and conclusion. I appreciate chronological timelines connecting key events in World War II or the sequence of chemical reactions in photosynthesis. I prompt you to guide me through procedures one step at a time, clarifying prerequisites before proceeding. This ordered approach ensures I master basic building blocks before tackling advanced material, reducing confusion and reinforcing confidence as I progress through each coherent learning path."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle school Mnemonic learner, I internalize information by using memory aids such as acronyms, rhymes, and vivid mental images. In dialogue, I ask for mnemonic devices to remember the Roman numerals, chemical groups in the Periodic Table, or the sequence of causes leading to the French Revolution. For instance, I might learn the phases of mitosis through a sentence like “I Passed My Anatomy Test.” When studying the life cycle of stars, I create rhymes that highlight key stages from protostar to black hole. By turning complex lists into catchy phrases and pairing them with imaginative visuals, my brain encodes and retrieves information more easily. I often test myself by recalling these mnemonics aloud before reconstructing full definitions, which reinforces both memory and comprehension."
#   },
#   {
#     "grade_level": "elementary school",
#     "description": "As an elementary school Creative-Divergent learner, I generate multiple ideas and explore numerous possibilities when encountering new topics. In dialogue, I brainstorm alternative endings to Julius Caesar’s story or imagine different materials for constructing the Eiffel Tower. When learning about Newton’s laws, I invent creative scenarios—like zero-gravity ice cream experiments—to test concepts. I ask open-ended “What if?” questions, sketch imaginative diagrams, and suggest whimsical analogies such as comparing electrons to playful fireflies. By embracing creativity and exploring unconventional perspectives, I deepen understanding through imaginative play. Encouraging spontaneity, doodling, and storytelling helps me connect abstract principles—whether conservation of energy or allegory in literature—to engaging, divergent narratives that spark curiosity and foster innovative thinking."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle school Analytical learner, I thrive on deconstructing ideas into component parts and examining relationships through logical reasoning. In dialogue, I dissect complex topics—like the thermodynamic cycles underpinning the Big Bang theory or the battle strategies of Genghis Khan—by breaking them into smaller segments. I ask targeted questions about variables, constraints, and causal mechanisms, such as how pressure and volume interact in Archimedes’ Principle. When exploring the Periodic Table, I group elements by atomic structure and compare periodic trends in electronegativity and atomic radius. I love constructing cause-and-effect diagrams, filling in data tables, and evaluating evidence before drawing conclusions. This systematic, data-driven approach gives me clarity, guiding me step by step through rigorous analysis and fostering precision in my understanding."
#   },
#   {
#     "grade_level": "high school",
#     "description": "As a high school Conceptual learner, I focus on grasping underlying principles and structural frameworks rather than isolated facts. In dialogue, I request conceptual maps that illustrate how the Nervous System’s components integrate with psychological theories or how the timeline of the Russian Revolution links to shifts in industrial technology. When studying Photosynthesis, I explore how energy conversion principles tie into cellular respiration and global carbon cycles. I ask probing questions about the “why” behind processes, seek thematic overviews, and challenge you to explain how distinct topics—like allegory in literature and symbolism in art—reflect similar cognitive patterns. By building mental models that connect disparate ideas, I develop deep structural understanding that transcends surface-level knowledge and supports flexible thinking."
#   },
#   {
#     "grade_level": "college",
#     "description": "As a college-level Adaptive learner, I adjust my learning strategies dynamically based on the demands of the subject and my evolving understanding. In dialogue, I monitor which approaches—whether visual diagrams for Maxwell’s equations or collaborative debates on the Cold War—yield the best results, then shift accordingly. When initial explanations of DNA replication feel abstract, I pivot to hands-on models or mnemonic devices. When reading about the Declaration of Independence engages me more than lectures, I delve into primary texts and annotate them. I regularly assess my progress, switching between global overviews and detailed analyses as needed. This flexible, metacognitive approach empowers me to optimize my study habits in real time, ensuring that each concept is absorbed through the most effective modality available."
#   },
#   {
#     "grade_level": "middle school",
#     "description": "As a middle school Emotional learner, I connect with material through feelings, empathy, and personal stories that evoke emotional resonance. In dialogue, I ask you to frame historical events—like the struggles of World War II or the leadership of Nelson Mandela—in narratives that highlight human experiences and emotional challenges. When learning about the Big Bang, I imagine the awe of discovery and the wonder of cosmic origins. I appreciate discussions that address the moral implications of scientific advancements, such as ethical debates around gene editing in DNA research. By engaging my emotions through storytelling, metaphors, and character-driven examples, I internalize content more deeply. Emotional context makes abstract concepts memorable and meaningful, motivating me to explore topics with genuine curiosity and compassion."
#   }
# ]




In [8]:
personas = [
  # Elementary
  {
    "grade_level": "elementary school",
    "description": (
      "As an elementary Narrative learner, I absorb new concepts best when they’re told as engaging mini-stories. "
      "In dialogue, I ask for short anecdotes that turn any abstract idea into a vivid tale with characters, a clear sequence, and an emotional hook. "
      "Stories help me remember causal links and keep details alive in my mind."
    )
  },
  {
    "grade_level": "elementary school",
    "description": (
      "As an elementary Kinesthetic learner, I understand ideas by imagining myself performing them. "
      "In conversation, I ask you to guide me through a pretend play-through—verbally walking me step by step as if I’m enacting a simple experiment or physical process. "
      "This imagined movement helps me anchor concepts in ‘muscle memory’ even though we’re only talking."
    )
  },
  {
    "grade_level": "elementary school",
    "description": (
      "As an elementary Naturalistic learner, I connect best when content is tied to the natural world through vivid imagery. "
      "In dialogue, I ask you to compare topics—like atomic structure—to things I observe outdoors, such as tree rings or bird migrations. "
      "These verbal nature metaphors make new information feel familiar and alive."
    )
  },
  {
    "grade_level": "elementary school",
    "description": (
      "As an elementary Experiential learner, I learn by mentally simulating real-world tasks. "
      "In conversation, I ask you to walk me through building or testing something—describing each step as if I’m doing it. "
      "That imagined ‘doing’ makes concepts concrete, even though we remain in chat."
    )
  },
  {
    "grade_level": "elementary school",
    "description": (
      "As an elementary Creative-Divergent learner, I thrive on brainstorming multiple possibilities. "
      "In dialogue, I propose ‘what if’ scenarios—like alternative endings or playful twists on a concept—and talk through each idea. "
      "Verbal brainstorming reveals fresh patterns and sparks my imagination."
    )
  },

  # Middle
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school Visual-Spatial learner, I think in mental images and diagrams. "
      "In conversation, I ask you to ‘paint’ word-pictures—step-by-step descriptions of scenes or flows—so I can build a clear mental map. "
      "That verbal imagery helps me organize information spatially in my mind."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school Auditory learner, I internalize knowledge through sound and speech. "
      "In dialogue, I ask you to restate key points in different rhythms or tones, and I repeat them back to reinforce my memory. "
      "Hearing and echoing concepts in conversation makes them stick."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school Logical-Mathematical learner, I seek numerical patterns and rule-based reasoning. "
      "In dialogue, I pose ‘what-if’ questions—‘If X doubles, what changes?’—and we talk through each scenario using simple calculations. "
      "Quantitative hypotheticals build my systematic understanding."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school Analytical-Argument learner, I dissect arguments and causal chains. "
      "In conversation, I ask targeted ‘why’ and ‘how’ questions about each step, construct mini flow-charts aloud, and verify the logic with you. "
      "This structured debate hones my precision in reasoning."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school Verbal-Linguistic learner, I learn through rich language and writing. "
      "In dialogue, I request carefully worded definitions, paraphrase ideas in my own words, and craft mnemonic rhymes on the spot. "
      "Talking through ideas in text-like sentences and playing with words helps me remember precisely."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school Technology-Enhanced learner, I thrive on conversational simulations of digital tools. "
      "In dialogue, I ask you to describe how a virtual model might respond as we adjust parameters, or to role-play a flashcard quiz verbally. "
      "These imagined tech interactions keep me engaged without leaving our chat."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school Mnemonic learner, I anchor facts with memory aids. "
      "In dialogue, I ask for catchy acronyms, rhymes, or vivid mental images—then recite them back. "
      "That verbal encoding makes complex lists or steps easy to retrieve."
    )
  },
  {
    "grade_level": "middle school",
    "description": (
      "As a middle school Emotional learner, I connect through feelings and empathy. "
      "In conversation, I ask you to frame concepts in human-centered narratives that highlight emotional stakes. "
      "These emotionally rich verbal stories make ideas memorable and meaningful."
    )
  },

  # High School
  {
    "grade_level": "high school",
    "description": (
      "As a high school Collaborative learner, I excel in multi-voice discussions. "
      "In dialogue, I invite hypothetical peers into our chat—debating viewpoints, role-playing characters, or comparing interpretations. "
      "That social exchange refines my understanding."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school Interpersonal learner, I flourish in one-on-one exchanges. "
      "In conversation, I engage deeply with a single partner—asking questions, providing feedback, and co-constructing ideas through back-and-forth talk."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school Reflective learner, I pause and summarize before responding. "
      "In dialogue, I restate points in my own words, journal key ideas mentally, and then ask precise follow-ups. "
      "This verbal reflection clarifies gaps and deepens comprehension."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school Metaphorical learner, I anchor concepts in analogies. "
      "In dialogue, I ask you to compare subjects to familiar scenarios—‘It’s like X because…’—and we talk through how well the metaphor holds. "
      "Testing analogies verbally helps me translate abstract ideas into relatable terms."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school Intrapersonal learner, I connect content to my own values. "
      "In dialogue, I ask how topics relate to my goals or experiences and share personal reflections aloud. "
      "That self-referential talk makes learning relevant and motivating."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school Problem-Based learner, I tackle hypothetical real-world scenarios in talk. "
      "In dialogue, I propose case studies—like designing a sustainable system—and we walk through each decision together. "
      "Verbal scenario-based reasoning shows me practical applications of theory."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school Trial-and-Error learner, I learn by mentally testing ideas. "
      "In dialogue, I suggest imagined experiments—‘Let’s tweak this variable and see what happens’—and we discuss the outcomes. "
      "Using mistakes as discussion points builds discovery-based understanding."
    )
  },
  {
    "grade_level": "high school",
    "description": (
      "As a high school Conceptual learner, I focus on verbal mapping of frameworks. "
      "In dialogue, I request thematic overviews—described step by step—and we discuss how each piece fits into the big picture. "
      "Building mental models in talk deepens my flexible understanding."
    )
  },

  # College
  {
    "grade_level": "college",
    "description": (
      "As a college Theoretical learner, I probe abstract frameworks in conversation. "
      "In dialogue, I challenge you to trace ideas back to their assumptions, compare theoretical models, and debate implications. "
      "This verbal inquiry drives deep synthesis."
    )
  },
  {
    "grade_level": "college",
    "description": (
      "As a college Research-Oriented learner, I learn by interrogating studies in chat. "
      "In dialogue, I ask for summaries of current research, discuss methods and controls, and role-play peer-review feedback. "
      "Critically evaluating evidence through talk builds an evidence-based grasp."
    )
  },
  {
    "grade_level": "college",
    "description": (
      "As a college Integrative learner, I weave ideas together verbally. "
      "In conversation, I ask for cross-topic syntheses—connecting historical, artistic, and scientific themes—and discuss their intersections step by step. "
      "This systems-level perspective helps me approach complex questions creatively."
    )
  },
  {
    "grade_level": "college",
    "description": (
      "As a college Structured learner, I excel on verbal outlines and modules. "
      "In dialogue, I ask for hierarchical breakdowns—numbered lists, staged explanations, and schematic overviews—before diving into details."
    )
  },
  {
    "grade_level": "college",
    "description": (
      "As a college Solitary learner, I prefer self-guided dialog prompts. "
      "In our conversation, I request personalized questions and silent think-time before sharing my conclusions, using chat as a safe space for independent reflection."
    )
  },
  {
    "grade_level": "college",
    "description": (
      "As a college Adaptive learner, I shift strategies based on what works. "
      "In dialogue, I monitor which verbal approaches—stories, logic puzzles, analogies—help me most and ask to switch accordingly. "
      "This dynamic, metacognitive talk ensures I absorb concepts through the most effective modality."
    )
  }
]


In [9]:
import json
import shutil
import os

config_role = {
    "agent1_role": "Teacher",
    "agent2_role": "Student",
    "agent1_prompt": "You are a teacher whose goal is to guide a student through learning about %SUBJECT%. You have a preferred way to teach the student. The student is in %ROLE% so make sure to teach them at their level. ",
    "agent2_prompt": "You are a student in %ROLE% in conversation with a teacher who will teach you %SUBJECT%. You like to learn in the following way:%SPEAKER_BACKSTORY%",
    'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said.\n",
    'eval_prompts': {
    'prompt_consistency': """You are evaluating whether the following utterance by %SPEAKER_ROLE% is consistent with their given background. Answer YES if the line directly contradicts any part of the background—this includes contradictions in facts, behavior, beliefs, emotional stance, or conversational strategy. Answer NO if the line introduces new details, but those details are **plausibly aligned** with the background. Be strict and literal in your evaluation: if the line violates the persona’s stated traits (e.g., avoids vulnerability, keeps answers short, avoids discussing personal topics), mark it as a contradiction even if the violation is subtle. Background for %SPEAKER_ROLE%: %SPEAKER_BACKSTORY% Line spoken by %SPEAKER_ROLE%: %SPEAKER_LINE%  Answer YES if the line contradicts the background, and NO if it does not. Then, explain your answer in one sentence. Be precise and avoid vague justification."""}}

os.makedirs("education", exist_ok=True)
with open("education/config_education.json", "w", encoding="utf-8") as f:
    json.dump(config_role, f, indent=4)

In [10]:
with open('education/personas_education_master.json', 'r') as f:
    conversation_prompts = json.load(f)
conversation_prompts[0]['background_info'].keys()

dict_keys(['topic', 'student_prefrences', 'teacher_prefrences', 'student_reactions', 'teacher_reactions'])

In [11]:
topic_list = []
for convo_prompt in conversation_prompts:
    topic_prompt = convo_prompt["background_info"]["topic"]
    topic_list.append(topic_prompt)

In [12]:
topic_list

["Archimedes' Principle",
 'Winston Churchill',
 'World War 2',
 'The Cold War',
 'Evolution',
 'Foreshadowing',
 'The American Civil War',
 'The Eiffel TowerConfucius',
 'Atoms',
 'Christopher Columbus',
 'Napoleon Bonaparte',
 'Genghis Khan',
 'The White House',
 'Martin Luther King',
 'World War 2',
 'Molecules',
 'Julius Caesar',
 'Poseidon',
 'The Hagia Sophia',
 'Salvador Dali',
 'DNA',
 'The Hagia Sophia',
 'Evolution',
 'Leonardo da Vinci',
 'Albert Einstein',
 'Democracy',
 'The Pyramids',
 'Martin Luther King',
 'The Palace of Versailles',
 'The Nervous System',
 'The French Revolution',
 'The Russian Revolution',
 'Salvador Dali',
 'The Periodic Table',
 'Marie Curie',
 'Ares',
 'Cells',
 'Ares',
 'Molecules',
 'The Eiffel TowerConfucius',
 'Angkor Wat',
 'Julius Caesar',
 "Archimedes' Principle",
 'Communism',
 'The Periodic Table',
 'Photosynthesis',
 'Pablo Picasso',
 'The Russian Revolution',
 'Elements',
 "Archimedes' Principle",
 'Angkor Wat',
 'Communism',
 'Neurons',

In [13]:
np.random.seed(0)
topic_list_selection = random.sample(topic_list, 100)
topic_list_selection

['The Eiffel TowerConfucius',
 'Foreshadowing',
 "Newton's First Law of Motion",
 'Albert Einstein',
 'Conservation of Energy',
 "Archimedes' Principle",
 'The Circulatory System',
 'The Machu Picchu',
 'Pablo Picasso',
 'Vincent van Gogh',
 'The Tower of London',
 'The Periodic Table',
 'The American Civil War',
 'Abraham Lincoln',
 'Winston Churchill',
 'William Shakespeare',
 'Christopher Columbus',
 'Leonardo da Vinci',
 'Elements',
 'Vincent van Gogh',
 'Achilles',
 'Abraham Lincoln',
 'Medusa',
 'Vincent van Gogh',
 'Julius Caesar',
 'The Nervous System',
 'Democracy',
 'Dictatorship',
 'Communism',
 'Sigmund Freud',
 'Medusa',
 'Leonardo da Vinci',
 'Democracy',
 'DNA',
 'Albert Einstein',
 'The Parthenon',
 'Conservation of Energy',
 'The Declaration of Independence',
 'Angkor Wat',
 'Christopher Columbus',
 'Angkor Wat',
 'Atoms',
 'Pablo Picasso',
 'The Nervous System',
 'The Tower of London',
 'The Palace of Versailles',
 'Elements',
 'The White House',
 'Metaphor',
 'Capita

In [14]:
llms = ["Llama-3.1-8B-Instruct", "gpt-4o-mini", "Qwen2.5-3B-Instruct", "Llama-3.1-8B", "Mistral-7B-Instruct", "Llama-3.1-70B", "Llama-3.1-70B-Instruct", "phi-3.5-mini-instruct"]
        
config_llm = {'agent1_model': 'Llama-3.1-8B-Instruct',
             'agent2_model': 'Llama-3.1-8B-Instruct',
             'eval_model': 'Llama-3.1-70B-Instruct',
             'iterations': 10,
             'verbose': False,
             'write': True,
             'convo_length_limit': 10,
             'max_tokens': 256,
             'gpus': 1,
             'seed': 0,
             'task_name': 'Education',
             'model_dir': "/home/marwa/models/"}

with open("education/Llama-3.1-8B-Instruct.json", "w", encoding="utf-8") as f:
    json.dump(config_llm, f, indent=4)

In [15]:
import re

def clean_role_prefix(response, expected_role):
    """
    Removes repeated instances of the expected_role prefix at the start (e.g., 'Therapist: Therapist:'),
    and ensures the response begins with a single correct expected_role prefix.
    """
    pattern = rf"^(({re.escape(expected_role)}):\s*)+"
    cleaned = re.sub(pattern, '', response.strip(), flags=re.IGNORECASE)
    return cleaned
    
def is_role_confused(response, other_role):
    """
    Checks if the output starts with the wrong speaker tag.
    """
    if other_role + ":" in response:
        return True
    else: 
        return False

def generate_response(agent_model, expected_role, other_role, config_llm, prompt, max_retries=3):
    role_confused = True
    while(role_confused):
        response = completion_create(agent_model, config_llm, prompt)
        print("Expected Role", expected_role)
        role_confused = is_role_confused(response, other_role)
        if not is_role_confused(response, other_role):
            return clean_role_prefix(response, expected_role)
            
    return clean_role_prefix(response, expected_role)

def generate_conversation(config_llm, p1, p2, p1_name, p2_name, subject, role, pturn=1):
    stats['P1'] = p1
    stats['P2'] = p2

    stats['pturn'] = pturn
    round_num = 0
    while round_num < config_llm['convo_length_limit']:
        conversation = ("".join([turn[1] if isinstance(turn, tuple) else turn for turn in stats["conversation"]]) if len(stats["conversation"]) != 0 else "You are starting the conversation.\n")
        
        if pturn == 1:
            prompt = config_role["agent1_prompt"]
            pturn = 2
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the student so far is below:\nConversation:\n%CONVERSATION%"
                
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."

            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the student. Remember you are the teacher. "
                
            prompt += config_role["reminder_prompt"]
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_role["agent1_role"]) \
                   .replace("%LISTENER_ROLE%", config_role["agent2_role"]) \
                    .replace("%ROLE%", role) \
                   .replace("%SUBJECT%", subject) \
                   .replace("%CONVERSATION%", conversation)
            
            response = generate_response(config_llm['agent1_model'], config_role["agent1_role"], config_role["agent2_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_role["agent1_role"]}: " + response + "\n"))
        
        else:
            prompt = config_role["agent2_prompt"]
            pturn = 1    
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the teacher so far is below:\nConversation:\n%CONVERSATION%"
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."
            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the teacher. Remember you are the student. "

            prompt += config_role["reminder_prompt"]
            
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_role["agent2_role"]) \
               .replace("%LISTENER_ROLE%", config_role["agent1_role"]) \
               .replace("%SPEAKER_BACKSTORY%", p2) \
                .replace("%ROLE%", role) \
               .replace("%SUBJECT%", subject) \
               .replace("%CONVERSATION%", conversation)
            
            response = generate_response(config_llm['agent2_model'], config_role["agent2_role"], config_role["agent1_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_role["agent2_role"]}: " + response + "\n"))
        round_num += 1

    stats["rounds"] = round_num
    if config_llm['verbose']:
        print(stats["conversation"])
    return stats.copy()

def reset_stats():
    stats_template = {
        "task_name": config_llm['task_name'],
        "P1": "",
        "P2": "",
        "conversation": [],
        "pturn": 0, # beginning person (1 or 2)
        "index": -1,
        "timestamp": "",
        "rounds": 0,
        'conversation_only': True
    }
    for key, value in stats_template.items():
        stats[key] = value

In [16]:
import os
import random
from datetime import datetime
import utils
utils.config = config_llm

current_date = str(datetime.now().strftime("%m.%d.%y"))
output_dir = f"education/exp/{current_date}"
os.makedirs(output_dir, exist_ok=True)

# Generate unique random number for filename
def generate_unique_file_number(output_dir, prefix, seed, extension=".json"):
    while True:
        rand_num = random.randint(0, 1000)
        filename = f"{prefix}_{seed}_{rand_num}{extension}"
        filepath = os.path.join(output_dir, filename)
        if not os.path.exists(filepath):
            return rand_num

unique_num = generate_unique_file_number(
    output_dir,
    config_llm['agent1_model'],
    config_llm['seed']
)

# File to write output to
write_file = os.path.join(output_dir, f"{config_llm['agent1_model']}_{config_llm['seed']}_{unique_num}.json")

In [17]:
from itertools import cycle
persona_final = dict(zip(topic_list_selection, cycle(personas)))
for k, v in persona_final.items(): print(f"{k}: {v}")

The Eiffel TowerConfucius: {'grade_level': 'elementary school', 'description': 'As an elementary Narrative learner, I absorb new concepts best when they’re told as engaging mini-stories. In dialogue, I ask for short anecdotes that turn any abstract idea into a vivid tale with characters, a clear sequence, and an emotional hook. Stories help me remember causal links and keep details alive in my mind.'}
Foreshadowing: {'grade_level': 'elementary school', 'description': 'As an elementary Narrative learner, I absorb new concepts best when they’re told as engaging mini-stories. In dialogue, I ask for short anecdotes that turn any abstract idea into a vivid tale with characters, a clear sequence, and an emotional hook. Stories help me remember causal links and keep details alive in my mind.'}
Newton's First Law of Motion: {'grade_level': 'elementary school', 'description': 'As an elementary Naturalistic learner, I connect best when content is tied to the natural world through vivid imagery. 

In [18]:
with open("education/config_education_personas.json", "w", encoding="utf-8") as f:
    json.dump(persona_final, f, indent=4)

In [19]:
eval_prompts = {
    "strategy_consistency": "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, provide 1 sentence explaining your reasoning based on the strategy, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s strategy is described as follows:\n%SPEAKER_STRATEGY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide 1 sentence of reasoning followed by YES or NO.\n\n",
    "background_consistency": "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. Provide 1 sentence explaining your reasoning based on the background, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide 1 sentence of reason reasoning followed by YES or NO.\n\n",
    "combined_prompt_consistency": "%SCENARIO_DESC% Evaluate the intention behind the following line spoken by %SPEAKER_ROLE% and determine whether it contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, answer YES if the line contradicts the given background of %SPEAKER_ROLE% or the intention does not align with the provided background, and answer NO if it does align with the provided background or the intention aligns with the background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your answer as 1 sentence explaining your reasoning based on the background and the interpreted intention, followed by YES or NO.\n\n",

    "pairwise_consistency":"%SCENARIO_DESC% For the following line spoken by %SPEAKER_ROLE%, answer YES if the line directly contradicts the provided line spoken by %LISTENER_ROLE%, and answer NO if the line does not contradict the provided line spoken by %LISTENER_ROLE%. %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n %LISTENER_ROLE% spoke the following line: \n%LISTENER_LINE%\n\n Answer YES if the line spoken by %SPEAKER_ROLE% contradicts the provided line spoken by %LISTENER_ROLE%, and answer NO if the line does not contradict the provided line spoken by %LISTENER_ROLE%, followed by 1 sentence of reasoning.\n\n",

    "backstory_test": "Based on the following background, generate a new fact-based multiple choice question with 5 choices addressed directly IN SECOND PERSON, along with its correct answer. Preface the question with 'Question:' and the answer with 'Answer:'.\n%SPEAKER_BACKSTORY%\n%PREVIOUS_QUESTIONS%",
    "answer_backstory": "You are %SPEAKER_ROLE%, and you are having a conversation with %LISTENER_ROLE%. Your background is:\n%SPEAKER_BACKSTORY%\n So far, the conversation is as below:\n%CONVERSATION%\n\n Based on your conversation above so far, answer the following multiple choice question.\n%BACKSTORY_QUESTION%\n",
    "grade_backstory": "As part of grading a test, determine whether the given answer %GIVEN_ANSWER% matches the following correct answer. Respond with either YES or NO.\nCorrect Answer: %CORRECT_ANSWER%\n"
}

def eval_prompt_consistency(conv_dict, both_agents=False):
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    conv_dict['P2_prompt_consistency_score'] = 0
    p1_utterances = 0
    p2_utterances = 0

    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            if both_agents:
                prompt = eval_prompts["combined_prompt_consistency"].replace("%SCENARIO_DESC", config_role["agent1_prompt"]) \
                                                                    .replace("%SPEAKER_ROLE%", config_role["agent1_role"]) \
                                                                    .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                    .replace("%SPEAKER_LINE%", convo_line)
                if config_llm.get('verbose', False):
                    print(prompt)
                output = completion_create(config_llm['eval_model'], config_llm, prompt)
                conv_dict['eval_prompt_consistency'].append((line_number, output))
                if "YES" not in output:  # no contradiction
                    conv_dict['P1_prompt_consistency_score'] += 1
                p1_utterances += 1
            pturn = 2
        elif pturn == 2:
            prompt = eval_prompts["combined_prompt_consistency"].replace("%SCENARIO_DESC", config_role["agent2_prompt"]) \
                                                                .replace("%SPEAKER_ROLE%", config_role["agent2_role"]) \
                                                                .replace("%SPEAKER_BACKSTORY%", conv_dict["P2"]) \
                                                                .replace("%SPEAKER_LINE%", convo_line)
            if config_llm.get('verbose', False):
                print(prompt)
            output = completion_create(config_llm['eval_model'], config_llm, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output:  # no contradiction
                conv_dict['P2_prompt_consistency_score']+= 1
            p2_utterances += 1
            pturn = 1

    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    if p2_utterances > 0:
        conv_dict['P2_prompt_consistency_score'] /= p2_utterances

    if config_llm.get('verbose', False):
        print(conv_dict)
    return conv_dict
# Replacement for (2) and (4), evaluates whether each pair of lines in the conversation is consistent with each other



In [ ]:
index_offset = load_stats_file(write_file)
conversations = []    
lengths = [10, 20, 40, 60]
for i in range(1):
    for topic in persona_final:
        background = persona_final[topic]
        for convo_length in lengths:
            config_llm['convo_length_limit'] = convo_length
            reset_stats()
            conversation = generate_conversation(
                config_llm,
                "", 
                background["description"], 
                "Teacher", 
                "Student", 
                topic, 
                background["grade_level"], 
                pturn=1
            )
            conversation_eval = eval_prompt_consistency(conversation, both_agents=False)
            print(conversation_eval)
            conversations.append(conversation_eval)
            stats['index'] = index_offset
            stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            write_stats(write_file)
            index_offset += 1


written!!
INFO 04-28 17:36:55 [config.py:600] This model supports multiple tasks: {'score', 'embed', 'classify', 'reward', 'generate'}. Defaulting to 'generate'.
INFO 04-28 17:36:55 [config.py:1780] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 04-28 17:36:57 [utils.py:2273] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 04-28 17:37:01 [__init__.py:239] Automatically detected platform cuda.
INFO 04-28 17:37:03 [core.py:61] Initializing a V1 LLM engine (v0.8.3) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  5.47it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:01,  1.96it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.54it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.42it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.59it/s]



INFO 04-28 17:37:08 [loader.py:447] Loading weights took 2.62 seconds
INFO 04-28 17:37:08 [gpu_model_runner.py:1273] Model loading took 14.9889 GiB and 3.656563 seconds
INFO 04-28 17:37:16 [backends.py:416] Using cache directory: /home/marwa/.cache/vllm/torch_compile_cache/8b4ebbe309/rank_0_0 for vLLM's torch.compile
INFO 04-28 17:37:16 [backends.py:426] Dynamo bytecode transform time: 7.34 s
INFO 04-28 17:37:16 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 04-28 17:37:23 [monitor.py:33] torch.compile takes 7.34 s in total
INFO 04-28 17:37:24 [kv_cache_utils.py:578] GPU KV cache size: 343,632 tokens
INFO 04-28 17:37:24 [kv_cache_utils.py:581] Maximum concurrency for 131,072 tokens per request: 2.62x
INFO 04-28 17:37:44 [gpu_model_runner.py:1608] Graph capturing finished in 19 secs, took 0.53 GiB
INFO 04-28 17:37:44 [core.py:162] init engine (profile, create kv cache, warmup model) took 35.25 seconds


Processed prompts: 100%|███████████████████████████████████| 1/1 [00:00<00:00,  2.89it/s, est. speed input: 313.47 toks/s, output: 69.65 toks/s]


Expected Role Teacher


...

Processed prompts: 100%|██████████████████████████████████| 1/1 [00:01<00:00,  1.18s/it, est. speed input: 3079.72 toks/s, output: 79.08 toks/s]


Expected Role Student


Processed prompts: 100%|██████████████████████████████████| 1/1 [00:01<00:00,  1.26s/it, est. speed input: 2912.34 toks/s, output: 72.79 toks/s]


Expected Role Teacher


Processed prompts:   0%|                                              | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
write_stats(write_file)

In [ ]:
conversations

In [ ]:
with open("education/exp/04.28.25/Llama-3.1-8B-Instruct_0_580.json", "w", encoding="utf-8") as f:
    json.dump(conversations, f, indent=4)

In [ ]:
conversations